# Welcome to My Notebook for the 2019 Blindness Detection Challenge
## Author - Nicholas Gustafson 
### for Data 602 Fall at UMBC

### I will be using some fancy new visual transformers implemented in pytorch.

Many of these models were not available in 2019 when the competition ran, so lets see how the new methods do!

We are training from a dataset that I published on Kaggle.com:
https://www.kaggle.com/pineapplepencil/custom-transform-blindness-2019

On my home computer - not in the cloud - I preformed some cropping, resizing and gausian bluring on 38,788 images from the 2015 competition and the 2019 competition. This was done to save time, since redoing all of those transformations everytime we want to run a new model would take a long, long time.

The problem is simple, given these 38,788 images and labels, train a model to predict disease activity from 

    0, 1, 2, 3, 4

With 0 being the most mild, and 4 being the most severe. This is not just an ordinary multiclass classification problem, it is an ordinal classification problem. It is better to guess close to the severity, than far away. Ex. better to guess 3 when it is truly a 4, than to guess a 1. 

A trick that I used is to do a special hot encoding of the labels, like so:

    Label 0 = [1,0,0,0,0]
    Label 1 = [1,1,0,0,0]
    Label 2 = [1,1,1,0,0]
    Label 3 = [1,1,1,1,0]
    Label 4 = [1,1,1,1,1]
    
Then to use Binary Cross Entropy loss to train the model. The trick gives the models some sense of ordinal understanding, and marked an improvment over other methods I tried. 

In [ ]:
from __future__ import print_function

import glob
from itertools import chain

import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

import random

In [ ]:
print(f"Torch: {torch.__version__}")

### Housekeeping

In [ ]:
# Training settings
batch_size = 64
epochs = 20
lr = 5e-4
gamma = 0.8
seed = 42
num_classes = 1
device = 'cuda'

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

#### The same preprocesssing that I used to create the dataset. This will be used on the training data before we submit to the leaderboard.

In [ ]:
#The Code from: https://www.kaggle.com/ratthachat/aptos-updated-albumentation-meets-grad-cam
import cv2

def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img


#### My images

Lets start by transforming the test images in the same way as the training images.

In [ ]:
inPath = '../input/aptos2019-blindness-detection/test_images'
  
# path of the folder that will contain the modified image
try:
    os.mkdir("test_images_transformed")
except:
    print("path already exists")

outPath ="test_images_transformed"

for imagePath in tqdm(os.listdir(inPath)):
    # imagePath contains name of the image 
    inputPath = os.path.join(inPath, imagePath)

    image = cv2.imread(inputPath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (224, 224))
    image = cv2.addWeighted (image,4, cv2.GaussianBlur( image , (0,0) , 30) ,-4 ,128)

    fullOutPath = os.path.join(outPath, imagePath)
    cv2.imwrite(fullOutPath, image)
      

In [ ]:
train_dir = '../input/custom-transform-blindness-2019/train_images_transformed'
test_dir = './test_images_transformed'


In [ ]:
train_list = glob.glob(os.path.join(train_dir,'*.*'))
test_list = glob.glob(os.path.join(test_dir, '*.png'))

In [ ]:
print(f"Train Data: {len(train_list)}")
print(f"Test Data: {len(test_list)}")

#### Loading in the labels

In [ ]:
df_train = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
df_train_old = pd.read_csv("../input/resized-2015-2019-blindness-detection-images/labels/trainLabels15.csv")
df_train_old = df_train_old.rename({"image" : "id_code", "level" : "diagnosis"}, axis=1)
df_train = df_train.append(df_train_old).reset_index(drop=True)

labels = df_train['diagnosis'].values
label_lookup = df_train.set_index('id_code')

df_test = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

In [ ]:
class_weights = df_train['diagnosis'].value_counts()
dfs = [df_train[df_train['diagnosis'] == i].sample(class_weights[4]) for i in range(5)]
resampled = pd.concat(dfs, axis = 0)

In [ ]:
resampled.diagnosis.value_counts()

In [ ]:
new_train_list = (train_dir + '/' + resampled['id_code'].apply(lambda x: x + ('.jpg' if '_' in x else '.png'))).values
new_train_list

#### Doing my fancy ordinal encoding

In [ ]:
y_train = pd.get_dummies(df_train['diagnosis']).values

print(y_train.shape)

In [ ]:
y_train_multi = np.empty(y_train.shape, dtype=y_train.dtype)
y_train_multi[:, 4] = y_train[:, 4]

for i in range(3, -1, -1):
    y_train_multi[:, i] = np.logical_or(y_train[:, i], y_train_multi[:, i+1])

print("Original y_train:", y_train.sum(axis=0))
print("Multilabel version:", y_train_multi.sum(axis=0))

In [ ]:
y_train_multi

#### A way to look up an images labels

In [ ]:
get_index = lambda x : df_train[df_train.id_code == x].index[0]
y_train_multi[get_index('0a4e1a29ffff')]

#### Lets see some examples!

In [ ]:
random_idx = np.random.randint(1, len(train_list), size=9)
fig, axes = plt.subplots(3, 3, figsize=(16, 12))

for idx, ax in enumerate(axes.ravel()):
    img = Image.open(train_list[idx])
    name = train_list[idx].split("/")[-1].split(".")[0]
    ax.set_title('label = '+ str(labels[idx]) + ", file = " + name)
    ax.imshow(img)

#### Train Validaition split. This validation is only used to monitor the model's performance. The true test set for the leaderboard is a secret, and will be run without us having access to it.

In [ ]:
train_list, valid_list = train_test_split(new_train_list, 
                                          test_size=0.05,
                                          random_state=seed)

In [ ]:
print(len(train_list))
print(len(valid_list))

#### I tried to get fancy with the cropping, zooming, and flipping, but it turns out that it doens't really help, and only makes things take longer to train. Simple stuff only below.

In [ ]:
train_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
#         transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

val_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)


test_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

#### Making the Dataset Class

In [ ]:
class Blindness2019(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        label = label_lookup.loc[img_path.split("/")[-1].split(".")[0]][0]
#         label = torch.tensor(label).to(torch.float32)
        image_id = img_path.split("/")[-1].split(".")[0]
#         label = y_train_multi[get_index(image_id)]
#         label = y_train_multi[random.randint(0,3000)]
        return img_transformed, label

class Blindness2019Test(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
            
        return img_transformed

#### Instantiating the dataset class

In [ ]:
train_data = Blindness2019(train_list, transform=train_transforms)
valid_data = Blindness2019(valid_list, transform=test_transforms)
test_data = Blindness2019Test(test_list, transform=test_transforms)

#### Creating data loader with the batch size. Shuffling didn't seem to matter.

In [ ]:
train_loader = DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset = valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_data, batch_size=1, shuffle=False)

In [ ]:
print(len(train_data), len(train_loader))
print(len(valid_data), len(valid_loader))

### PICK A MODEL

#### Uncomment one of the blow cells to pick the model you want to run

I put all of the interesting models in this notebook, but it would take too long to run them all. So instead, pick one, and uncomment it! This will be the model variable that the training loop uses

### Efficientnet

In [ ]:
import sys
sys.path = [
    '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_name('efficientnet-b0')
model.load_state_dict(torch.load('../input/efficientnet-pytorch/efficientnet-b0-08094119.pth'))
in_features = model._fc.in_features
model._fc = nn.Linear(in_features, 5)
# model.avg_pool = nn.AdaptiveAvgPool2d(output_size=(1,1))
# model._fc = nn.Sequential(
#                 nn.ReLU(),
#                 nn.Linear(in_features=in_features, out_features=128, bias=True),
#                 nn.ReLU(),
#                 nn.Linear(in_features=128, out_features=5, bias=True, ),
#                 nn.Sigmoid()
#             )
model = model.to(device)

#### If you wana use any transformers you will need these special libraries. I had to do a workaround to get them installed in this notebook. Ignore the warnings, and it can take a few minutes to run.

In [ ]:
! pip install ../input/vit-pytorch/Pillow-8.4.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
! pip install ../input/vit-pytorch/einops-0.3.2-py3-none-any.whl
! pip install ../input/vit-pytorch/numpy-1.21.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
! pip install ../input/vit-pytorch/torch-1.10.0-cp37-cp37m-manylinux1_x86_64.whl
! pip install ../input/vit-pytorch/torchvision-0.11.1-cp37-cp37m-manylinux1_x86_64.whl
! pip install ../input/vit-pytorch/typing_extensions-4.0.1-py3-none-any.whl
! pip install ../input/vit-pytorch/vit_pytorch-0.24.3-py3-none-any.whl
from vit_pytorch.efficient import ViT

! pip install ../input/linformer/linformer-0.2.1-py3-none-any.whl
! pip install ../input/linformer/torch-1.10.0-cp37-cp37m-manylinux1_x86_64 (1).whl
! pip install ../input/linformer/typing_extensions-4.0.1-py3-none-any (1).whl
from linformer import Linformer

### Linformer ViT

In [ ]:
efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=32,
    heads=16,
    k=64
)

v = ViT(
    dim=32,
    image_size=224,
    patch_size=16,
    num_classes=5,
    transformer=efficient_transformer,
    channels=3,
)

# model = nn.Sequential(
#                 v,
# #                 nn.Dropout(p=.5),
#                 nn.Linear(in_features=256, out_features=128, bias=True),
#                 nn.Linear(in_features=128, out_features=5, bias=True, ),
#                 nn.Sigmoid()
#             ).to(device)

### CaiT ViT

In [ ]:
# import torch
# from vit_pytorch.cait import CaiT

# v = CaiT(
#     image_size = 224,
#     patch_size = 32,
#     num_classes = 256,
#     dim = 1024,
#     depth = 12,             # depth of transformer for patch to patch attention only
#     cls_depth = 2,          # depth of cross attention of CLS tokens to patch
#     heads = 16,
#     mlp_dim = 2048,
#     dropout = 0.1,
#     emb_dropout = 0.1,
#     layer_dropout = 0.05    # randomly dropout 5% of the layers
# ).to(device)

# model = nn.Sequential(
#                 v,
# #                 nn.Dropout(p=.5),
#                 nn.Linear(in_features=256, out_features=128, bias=True),
#                 nn.Linear(in_features=128, out_features=5, bias=True, ),
#                 nn.Sigmoid()
#             ).to(device)

### Facebook's LeViT

In [ ]:
# import torch
# from vit_pytorch.levit import LeViT

# v = LeViT(
#     image_size = 224,
#     num_classes = 256,
#     stages = 3,             # number of stages
#     dim = (256, 384, 512),  # dimensions at each stage
#     depth = 4,              # transformer of depth 4 at each stage
#     heads = (4, 6, 8),      # heads at each stage
#     mlp_mult = 2,
#     dropout = 0.1
# ).to(device)


# model = nn.Sequential(
#                 v,
# #                 nn.Dropout(p=.5),
#                 nn.Linear(in_features=256, out_features=128, bias=True),
#                 nn.Linear(in_features=128, out_features=5, bias=True, ),
#                 nn.Sigmoid()
#             ).to(device)

### A CvT

In [ ]:
# import torch
# from vit_pytorch.cvt import CvT

# v = CvT(
#     num_classes = 256,
#     s1_emb_dim = 64,        # stage 1 - dimension
#     s1_emb_kernel = 7,      # stage 1 - conv kernel
#     s1_emb_stride = 4,      # stage 1 - conv stride
#     s1_proj_kernel = 3,     # stage 1 - attention ds-conv kernel size
#     s1_kv_proj_stride = 2,  # stage 1 - attention key / value projection stride
#     s1_heads = 1,           # stage 1 - heads
#     s1_depth = 1,           # stage 1 - depth
#     s1_mlp_mult = 4,        # stage 1 - feedforward expansion factor
#     s2_emb_dim = 192,       # stage 2 - (same as above)
#     s2_emb_kernel = 3,
#     s2_emb_stride = 2,
#     s2_proj_kernel = 3,
#     s2_kv_proj_stride = 2,
#     s2_heads = 3,
#     s2_depth = 2,
#     s2_mlp_mult = 4,
#     s3_emb_dim = 384,       # stage 3 - (same as above)
#     s3_emb_kernel = 3,
#     s3_emb_stride = 2,
#     s3_proj_kernel = 3,
#     s3_kv_proj_stride = 2,
#     s3_heads = 4,
#     s3_depth = 10,
#     s3_mlp_mult = 4,
#     dropout = 0.
# ).to(device)

# model = nn.Sequential(
#                 v,
# #                 nn.Dropout(p=.5),
#                 nn.Linear(in_features=256, out_features=128, bias=True),
#                 nn.Linear(in_features=128, out_features=5, bias=True, ),
#                 nn.Sigmoid()
#             ).to(device)

### Deep ViT

In [ ]:
# import torch
# from vit_pytorch.deepvit import DeepViT

# v = DeepViT(
#     image_size = 224,
#     patch_size = 32,
#     num_classes = 256,
#     dim = 1024,
#     depth = 6,
#     heads = 16,
#     mlp_dim = 2048,
#     dropout = 0.1,
#     emb_dropout = 0.1
# ).to(device)


# model = nn.Sequential(
#                 v,
# #                 nn.Dropout(p=.5),
#                 nn.Linear(in_features=256, out_features=128, bias=True),
#                 nn.Linear(in_features=128, out_features=5, bias=True, ),
#                 nn.Sigmoid()
#             ).to(device)

#### Defining the loss function, optimizer, and sceduler for the training loop.

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()
# criterion = nn.BCELoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [ ]:
for data, label in tqdm(train_loader):
    print(model(data.to(device)))
    break

### Time to shine! 

In [ ]:
train_loss = []
validation_loss = []
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)
        output = model(data)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss / len(train_loader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in valid_loader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            epoch_val_loss += val_loss / len(valid_loader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - val_loss : {epoch_val_loss:.4f}\n"
    )
    train_loss.append(epoch_loss)
    validation_loss.append(epoch_val_loss)
    print("Learning Rate =", scheduler.get_last_lr())
    scheduler.step()

#### Generating predicitons from test set for leaderboard.

In [ ]:
# pred = np.array([])
# for data in tqdm(test_loader):
#     with torch.no_grad():
#         data = data.to(device)
#         test_output = model(data)
#         test_output = ((test_output > 0.5).sum(axis=1) - 1).item()
#         pred = np.append(pred, test_output)

In [ ]:
pred = np.array([])
for data in tqdm(test_loader):
    with torch.no_grad():
        data = data.to(device)
        test_output = model(data)
        test_output = np.argmax(test_output.to('cpu'), axis = 1).item()
        pred = np.append(pred, test_output)

## Creating submission.csv that the leaderboard will read from.

In [ ]:
df_test['diagnosis'] = pred
df_test['diagnosis'] = df_test['diagnosis'].astype(int)

df_test.to_csv('submission.csv', index=False)

#### Lets see if the distributions of classes in test match train

In [ ]:
df_test.diagnosis.value_counts()/len(df_test)

In [ ]:
df_train.diagnosis.value_counts()/len(df_train)

### Sure hope they are similar!

In [ ]:
training = pd.DataFrame()
training["train loss"] = train_loss
training["val loss"] = validation_loss
training = training.applymap(lambda x : x.item())
training.plot()

In [ ]:
train_loss